In [2]:
import os
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation



In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 0.2,shear_range = 0.2,
    zoom_range = 0.2,width_shift_range = 0.2,
    height_shift_range = 0.2, validation_split = 0.2)

In [4]:




train_data= train_datagen.flow_from_directory(r'C:\Users\begum\Downloads\drowsiness_data\train',
                                target_size = (80,80), batch_size = 8, 
                                class_mode = 'categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\begum\Downloads\drowsiness_data\test',
                                target_size = (80,80), batch_size = 8, 
                                class_mode = 'categorical', subset='validation')




Found 64564 images belonging to 2 classes.
Found 838 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\begum\Downloads\drowsiness_data\test',
                                target_size=(80,80), batch_size =8, class_mode='categorical')

Found 4195 images belonging to 2 classes.


In [6]:
bmodel = InceptionV3(include_top = False, weights = 'imagenet', 
                     input_tensor = Input(shape = (80,80,3)))
hmodel = bmodel.output

In [7]:
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation = 'relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation = 'softmax')(hmodel)
model = Model(inputs = bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False


In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [9]:

checkpoint = ModelCheckpoint(os.path.join("models", "model.h5"),
                            monitor = 'val_loss', save_best_only = True, verbose = 3)

earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, 
                          verbose= 3, restore_best_weights = True)


learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )
callbacks = [checkpoint, earlystop, learning_rate]

In [10]:
callbacks = [checkpoint, earlystop, learning_rate]



model.compile(optimizer = 'Adam', 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])


model.fit_generator(train_data,steps_per_epoch = train_data.samples// 8,
                   validation_data = validation_data,
                   validation_steps = validation_data.samples// 8,
                   callbacks = callbacks,
                    epochs = 6)

C:\Users\begum\AppData\Local\Temp\ipykernel_6164\2511976908.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch = train_data.samples// 8,


Epoch 1/6
8070/8070 [==============================] - ETA: 0s - loss: 0.1869 - accuracy: 0.9264
Epoch 1: val_loss improved from inf to 0.36868, saving model to models\model.h5


C:\Users\begum\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8070/8070 [==============================] - 2632s 325ms/step - loss: 0.1869 - accuracy: 0.9264 - val_loss: 0.3687 - val_accuracy: 0.8510 - lr: 0.0010
Epoch 2/6
8070/8070 [==============================] - ETA: 0s - loss: 0.1632 - accuracy: 0.9377
Epoch 2: val_loss improved from 0.36868 to 0.33479, saving model to models\model.h5
8070/8070 [==============================] - 4412s 547ms/step - loss: 0.1632 - accuracy: 0.9377 - val_loss: 0.3348 - val_accuracy: 0.8690 - lr: 0.0010
Epoch 3/6
8070/8070 [==============================] - ETA: 0s - loss: 0.1536 - accuracy: 0.9412
Epoch 3: val_loss did not improve from 0.33479
8070/8070 [==============================] - 2229s 276ms/step - loss: 0.1536 - accuracy: 0.9412 - val_loss: 0.3513 - val_accuracy: 0.8834 - lr: 0.0010
Epoch 4/6
8070/8070 [==============================] - ETA: 0s - loss: 0.1521 - accuracy: 0.9427
Epoch 4: val_loss improved from 0.33479 to 0.30258, saving model to models\model.h5
8070/8070 [==============================

In [11]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\begum\AppData\Local\Temp\ipykernel_6164\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.12640027701854706
0.9503593444824219


In [12]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\begum\AppData\Local\Temp\ipykernel_6164\4081756742.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.3161574900150299
0.9045345783233643


In [13]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\begum\AppData\Local\Temp\ipykernel_6164\3655471885.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.12640027701854706
0.9503593444824219
